Base에서 실행

# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## 1) Dataload Reuters news  data

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import reuters

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(test_split=0.2)

category_list = []
for i in range(46):
    category_list.append(0)
print(len(category_list))

for i in train_labels:
    category_list[i] += 1

extracted_index = []

for i, v in enumerate(category_list):
    if v >= 100:
#         print(i, v)
        extracted_index.append(i)


extracted_train_data = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_data.append(train_data[i])

extracted_test_data = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_data.append(test_data[i])

extracted_train_labels = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_labels.append(train_labels[i])

extracted_test_labels = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_labels.append(test_labels[i])

word_to_index = reuters.get_word_index()

index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key # pad, sos, unk, NULL 포함

for index, token in enumerate(("<pad>", "<sos>", "<unk>")): # index=3은 NULL
    index_to_word[index] = token

#문장 추출
to_txt=[]
for i in range(7828):
  
    to_txt.append(' '.join([index_to_word[index] for index in extracted_train_data[i]])[6:])
    
for i in range(1934):
    to_txt.append(' '.join([index_to_word[index] for index in extracted_test_data[i]])[6:])

46


In [5]:
y=extracted_train_labels+extracted_test_labels

encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:7828]
y_test=label[7828:]

In [6]:
# 방법2) 케라스 내장함수 활용
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

## 2) textcuboid 분류

In [7]:
textcuboid=np.load('./1-Channel textcuboid_reuters(elmo).npy')

In [8]:
textcuboid_test=np.load('./1-Channel textcuboid_test_reuters(elmo).npy')

In [9]:
import keras
from tensorflow.keras import layers

In [10]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [11]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model


input_layer = Input(shape=(252, 256))
conv1 = Conv1D(256, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(256, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(12, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)



In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 252, 256)]        0         
                                                                 
 conv1d (Conv1D)             (None, 252, 256)          65792     
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 12)                3084      
                                                             

In [14]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

In [15]:
history = model.fit(x_train, y_train,callbacks=callbacks, epochs=20,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/20
27/27 [==============================] - 2s 35ms/step - loss: 1.6530 - accuracy: 0.5593 - val_loss: 0.9386 - val_accuracy: 0.7400 - lr: 0.0010
Epoch 2/20
27/27 [==============================] - 1s 19ms/step - loss: 0.8708 - accuracy: 0.7639 - val_loss: 0.7186 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 3/20
27/27 [==============================] - 1s 22ms/step - loss: 0.7050 - accuracy: 0.8143 - val_loss: 0.6335 - val_accuracy: 0.8240 - lr: 0.0010
Epoch 4/20
27/27 [==============================] - 1s 22ms/step - loss: 0.6290 - accuracy: 0.8295 - val_loss: 0.5871 - val_accuracy: 0.8410 - lr: 0.0010
Epoch 5/20
27/27 [==============================] - 1s 19ms/step - loss: 0.5590 - accuracy: 0.8515 - val_loss: 0.5674 - val_accuracy: 0.8510 - lr: 0.0010
Epoch 6/20
27/27 [==============================] - 1s 22ms/step - loss: 0.5040 - accuracy: 0.8682 - val_loss: 0.5535 - val_accuracy: 0.8630 - lr: 0.0010
Epoch 7/20
27/27 [==============================] - 1s 21ms/step - loss: 0.4

In [16]:
model.evaluate(textcuboid_test,one_hot_test_labels)

61/61 [==============================] - 0s 3ms/step - loss: 0.5463 - accuracy: 0.8790


[0.5462568998336792, 0.8790072202682495]

In [17]:
from keras.models import load_model
model = load_model('best_model.h5')
model.evaluate(textcuboid_test,one_hot_test_labels)

61/61 [==============================] - 0s 3ms/step - loss: 0.5160 - accuracy: 0.8744


[0.516044020652771, 0.8743536472320557]

In [32]:
np.average([88.1,87.6,88.1])

87.63333333333333